In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk

/projects/nanopore-working/rojin/nanoraw-signalAlign-nanopolish/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
embedding = pd.read_table('glove.twitter.27B.25d.txt', sep = " ", header= None)

In [3]:
embedding = embedding.dropna()

In [4]:
len(embedding[0])

1193508

In [8]:
embedding_matrix = embedding[list(range(1,26))].as_matrix()

In [9]:
# editing the input sentences
training_set = pd.read_csv('train.csv', encoding="ISO-8859-1")
training_set.head(3)

,handle,tweet
0,realDonaldTrump,"In Tampa, Florida- thank you to all of our out..."
1,realDonaldTrump,Poll: @realDonaldTrump vs. @HillaryClinton amo...
2,HillaryClinton,Obama on whether Trump could be trusted with U...


In [10]:
training_set['handle']=training_set['handle'].map({'HillaryClinton':1,'realDonaldTrump':0})

In [11]:
training_set.head(3)

,handle,tweet
0,0,"In Tampa, Florida- thank you to all of our out..."
1,0,Poll: @realDonaldTrump vs. @HillaryClinton amo...
2,1,Obama on whether Trump could be trusted with U...


In [12]:
# I adapted this from another a site, need to cite it here
def remove_by_regex(tweets, regexp):
    tweets.loc[:, "tweet"].replace(regexp, "", inplace=True)
    return tweets

def remove_special_chars(tweets):  # it unrolls the hashtags to normal words
    for remove in map(lambda r: re.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                 "@", "%", "^", "*", "(", ")", "{", "}",
                                                                 "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                 "!", "?", ".", "'",
                                                                 "--", "---", "#", "\n" , "—@", "→"]):
        tweets.loc[:, "tweet"].replace(remove, "", inplace=True)
    return tweets

def remove_things(tweets):
    remove_by_regex(tweets, re.compile(r"http.?://[^\s]+[\s]?"))
    remove_by_regex(tweets, re.compile(r"@[^\s]+[\s]?"))
    remove_by_regex(tweets, re.compile(r"\s?[0-9]+\.?[0-9]*"))
    remove_special_chars(tweets)
    return tweets

In [13]:
remove_things(training_set).head(3)

,handle,tweet
0,0,In Tampa Florida thank you to all of our outst...
1,0,Poll vs among white Evangelicals
2,1,Obama on whether Trump could be trusted with U...


In [14]:
#tokenize
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize
def tok(tweets):
    p = []
    for i in tweets['tweet']:
        tokenizer = RegexpTokenizer(r'\w+')
        p.append(tokenizer.tokenize(i))
    tweets['tok_tweet'] = p
    return tweets

In [15]:
tok(training_set).head(3)

,handle,tweet,tok_tweet
0,0,In Tampa Florida thank you to all of our outst...,"[In, Tampa, Florida, thank, you, to, all, of, ..."
1,0,Poll vs among white Evangelicals,"[Poll, vs, among, white, Evangelicals]"
2,1,Obama on whether Trump could be trusted with U...,"[Obama, on, whether, Trump, could, be, trusted..."


In [16]:
#stemmer
sno = nltk.stem.SnowballStemmer('english')
def stemm(tweets):
    p = []
    for i in tweets['tok_tweet']:
        q = []
        for j in i:
            q.append(sno.stem(j))
        p.append(q)
    tweets['tok_tweet'] = p
    return tweets

In [17]:
stemm(training_set).head(3)

,handle,tweet,tok_tweet
0,0,In Tampa Florida thank you to all of our outst...,"[in, tampa, florida, thank, you, to, all, of, ..."
1,0,Poll vs among white Evangelicals,"[poll, vs, among, white, evangel]"
2,1,Obama on whether Trump could be trusted with U...,"[obama, on, whether, trump, could, be, trust, ..."


In [18]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /soe/rojinsafavi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords
def stop_words_eliminatin(tweets):
    p = []
    stop_words = set(stopwords.words('english'))
    for i in tweets['tok_tweet']:
        q = []
        for j in i:
            if j not in stop_words:
                q.append(j)
        p.append(q)
        
    tweets['tok_tweet'] = p
    return tweets

In [20]:
stop_words_eliminatin(training_set).head(3)

,handle,tweet,tok_tweet
0,0,In Tampa Florida thank you to all of our outst...,"[tampa, florida, thank, outstand, volunt, want..."
1,0,Poll vs among white Evangelicals,"[poll, vs, among, white, evangel]"
2,1,Obama on whether Trump could be trusted with U...,"[obama, whether, trump, could, trust, us, nucl..."


In [21]:
# add tokenized sentece length:
P = []
for i in training_set['tok_tweet']:
    length = len(i)
    P.append(length)
training_set['length'] = P

In [22]:
# number of words in our dict:
count = {}
for i in training_set['tok_tweet']:
    for j in i:
        if j not in count:
            count[j] = 1
        else:
            count[j] += 1

In [23]:
# find the index of words for the sentence from the embedding
word_indexes = {}

for i,j in enumerate(embedding[0]):
    if j not in word_indexes:
        word_indexes[str(j)] = i
    else:
        print(i,j)
    
for i,j in enumerate(count.keys()):
    if j not in word_indexes:
        word_indexes[str(j)] = len(word_indexes) +1

In [25]:
h = np.asmatrix([25*[0]]*((len(set(count.keys()) - set(embedding[0])))))

embedding_matrix_with_zero = np.concatenate((embedding_matrix,
                                             h)).astype(np.float32)

In [26]:
embedding_matrix_with_zero.shape

(1195727, 25)

In [27]:
k = []
for i in training_set['tok_tweet']:
    l = []
    for j in i:
        l.append(word_indexes[j])
    k.append(l)
training_set['word_ids'] = k

In [28]:
training_set.head(3)

,handle,tweet,tok_tweet,length,word_ids
0,0,In Tampa Florida thank you to all of our outst...,"[tampa, florida, thank, outstand, volunt, want...",7,"[14387, 3780, 336, 1193509, 668240, 128, 1193510]"
1,0,Poll vs among white Evangelicals,"[poll, vs, among, white, evangel]",5,"[9836, 916, 8634, 811, 469675]"
2,1,Obama on whether Trump could be trusted with U...,"[obama, whether, trump, could, trust, us, nucl...",10,"[1380, 4527, 28622, 296, 978, 290, 11639, 1604..."
